<a href="https://colab.research.google.com/github/sajjad-yazdanparast/sentiment_analysis/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from gensim.models import KeyedVectors
import pandas as pd 
import numpy as np 

In [ ]:
# def gunzip(file_path,output_path):
#     with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
#         shutil.copyfileobj(f_in, f_out)
# gunzip('/content/drive/My Drive/parto tech/ GoogleNews-vectors-negative300.bin.gz','/content/drive/My Drive/parto tech/word2vec.bin')

In [3]:
model = KeyedVectors.load_word2vec_format(open('/content/drive/My Drive/parto tech/sentiment_analysis/word2vec.bin','rb'), binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [62]:
train = pd.read_csv('/content/drive/My Drive/parto tech/sentiment_analysis/train.csv') 
test = pd.read_csv('/content/drive/My Drive/parto tech/sentiment_analysis/test.csv')

In [63]:
def remove_atsign (text) :
  return text.replace('@user',' ')

In [64]:
train['tweet'] = train['tweet'].apply(remove_atsign)

In [75]:
def sentence2vec(row,w2v) :
  vector = np.zeros((1,300)) 
  words = row.split()
  count = 0 
  for word in words :
    try :
      vector += w2v[word].reshape((1,300))
      count +=1 
    except KeyError :
      continue 
  if count != 0 :
    return vector/count
  return vector

In [76]:
vectors = np.concatenate([sentence2vec(z,model) for z in train['tweet']])

In [77]:
np.shape(vectors)

(31962, 300)

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [48]:
train.head()

,id,label,tweet,vector
0,1,0,when a father is dysfunctional and is so se...,"[[0.0710915958180147, 0.03790642233455882, 0.0..."
1,2,0,thanks for #lyft credit i can't use cause ...,"[[0.03919534122242647, 0.015222886029411764, 0..."
2,3,0,bihday your majesty,"[[0.16959635416666666, 0.07747395833333333, 0...."
3,4,0,#model i love u take with u all the time in ...,"[[-0.04364885602678571, 0.021353585379464284, ..."
4,5,0,factsguide: society now #motivation,"[[-0.042205810546875, -0.015838623046875, -0.0..."


In [52]:
train.drop('tweet',axis=1,inplace=True)
train.head()

,id,label,vector
0,1,0,"[[0.0710915958180147, 0.03790642233455882, 0.0..."
1,2,0,"[[0.03919534122242647, 0.015222886029411764, 0..."
2,3,0,"[[0.16959635416666666, 0.07747395833333333, 0...."
3,4,0,"[[-0.04364885602678571, 0.021353585379464284, ..."
4,5,0,"[[-0.042205810546875, -0.015838623046875, -0.0..."


In [57]:
train['vector']

0        [[0.0710915958180147, 0.03790642233455882, 0.0...
1        [[0.03919534122242647, 0.015222886029411764, 0...
2        [[0.16959635416666666, 0.07747395833333333, 0....
3        [[-0.04364885602678571, 0.021353585379464284, ...
4        [[-0.042205810546875, -0.015838623046875, -0.0...
                               ...                        
31957    [[-0.082061767578125, 0.014892578125, 0.008117...
31958    [[0.056592858355978264, 0.05500063688858696, 0...
31959    [[0.03876906174879808, 0.0005563589242788461, ...
31960    [[0.035481770833333336, 0.05859375, 0.06917317...
31961    [[0.01131591796875, -0.03782958984375, 0.06674...
Name: vector, Length: 31962, dtype: object

In [79]:
from sklearn.model_selection import train_test_split
x_train,x_test , y_train,y_test = train_test_split(vectors,train['label'],random_state=101)

In [83]:
print(len(train[train['label']==0]))
print(len(train[train['label']==1]))
print(len(train[train['label']==0])/len(train[train['label']==1]))

29720
2242
13.256021409455842


In [92]:
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
# model_ = SGDClassifier()
# model_ = XGBClassifier()
model_ = RandomForestClassifier()
model_.fit(x_train,y_train)
model_.score(x_test,y_test)

0.9486922788136654